In [ ]:
import requests                # Include HTTP Requests module
from bs4 import BeautifulSoup  # Include BS web scraping module
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey, select
from datetime import date, datetime, timedelta
import re

In [ ]:
engine = create_engine('sqlite:///HoopStat.db')
conn = engine.connect()
metadata = MetaData(bind=None)
games = Table('games', metadata, autoload = True, autoload_with = engine)
conferences = Table('conferences', metadata, autoload = True, autoload_with = engine)
teams = Table('teams', metadata, autoload = True, autoload_with = engine)

In [ ]:
stmt = select([conferences.c.id])
allConf = conn.execute(stmt).fetchall()
allConf = [value for value, in allConf]
allConf

In [ ]:
def getGamesForConf(allTms):
    gamesDF = pd.DataFrame()
    year = '2019-20'
    for team in allTms:
    #team = 'Baruch'
        teamPage = 'https://d3hoops.prestosports.com/teams/' + team + '/Men/' + year + '/index'
        home = ''
        away = ''
        r = requests.get(teamPage)
        soup = BeautifulSoup(r.text, "html.parser")
        for i in soup.find_all('td'):#, attrs = {'valign': 'top'}):
            if i.findChild('a'):
                if '/teams/' in i.findChild('a')['href']:
                    opp = i.findChild('a')['href'][1:].split('/')[1]
                    HA = re.sub('[\W_]', '',i.text)[0:2]
                if '/seasons/' in i.findChild('a')['href']:
                    gameID = i.findChild('a')['href'].replace('//','/').split('/')[5].split('?')[0].replace('.xml','')
                    try:
                        if HA == 'at':
                            home, away = opp, team
                        else:
                            home, away = team, opp
                        home = home.replace(' ','_')
                        away = away.replace(' ','_')
                    except:
                        print(teamPage)
                    gamesDF = pd.concat([gamesDF,pd.DataFrame([[gameID,home,away]],columns=['id','home','away'])])
        #games[:25]
        #soup.prettify()
    return gamesDF
    #gamesDF.head(50)

In [ ]:
for conf in allConf:
    stmt = select([teams.c.id]).where(teams.columns.conf_id==conf)
    allTms = conn.execute(stmt).fetchall()
    allTms = [value for value, in allTms]
    gamesDF = getGamesForConf(allTms)
    gamesDF.to_sql('temp_table', engine, if_exists='replace')
    sql = """
    INSERT OR IGNORE INTO games(id,home,away)
    SELECT
    id,home,away
    FROM temp_table AS tt 
    WHERE true
    """
    try:
        conn.execute(sql)
    except:
        print(conf)

In [ ]:
pd.read_sql_query('SELECT \
g.id AS GameID \
,h.name AS HomeTeam \
,h.id AS HomeID \
,hc.name AS HomeConference \
,a.name AS AwayTeam \
,a.id AS AwayID \
,ac.name AS AwayConference \
FROM games g \
JOIN teams h ON g.home = h.id \
JOIN teams a ON g.away = a.id \
JOIN conferences hc ON h.conf_id = hc.id \
JOIN conferences ac ON a.conf_id = ac.id',conn)

In [ ]:
conn.close()